# Creates a Yelp POSTGRESSQL database
 - Uses the open source Yelp dataset and inserts json files into SQL tables

## Table of contents 
- importing json into python using pandas tools
- importing json to postgresql database

In [2]:
import pandas as pd

In [23]:
import json 
import pandas as pd 
from pandas.io.json._normalize import json_normalize,nested_to_record #flattening json in pandas df

from re import sub

In [3]:
yelp_data_path = '/Users/michaelkranz/Documents/restaurant-app/data/yelp_dataset/'

yelp_json_filenames = {"tips":'yelp_academic_dataset_tip.json',"reviews":'yelp_academic_dataset_review.json',
"business_info":'yelp_academic_dataset_business.json',"user":'yelp_academic_dataset_user.json'}

In [130]:
#%%timeit
with open(yelp_data_path+yelp_json_filenames['business_info']) as json_file:
    json_list = json_file.readlines()
json_data = [json.loads(json_line) for json_line in json_list] 
json_df = pd.DataFrame(json_data)

In [114]:
def convert_to_json_string_records(series):
    '''
    converts a series of dicts into a stringified 
    set of json records (which is necessary format for json storage)
    for example single quotes are replaced by double quotes
    and the dict is converted into a single quote string.
    '''
    if series.apply(lambda x: x is None or type(x) is dict).all():
       json_string_records = series.apply(json.dumps) 
    else:
        print("the series {} is already stringified so were not converted are not dicts so were not converted".format(series.name))
        json_string_records = series
    return json_string_records

In [115]:
json_df['attributes'] = convert_to_json_string_records(json_df['attributes'])
json_df['hours'] = convert_to_json_string_records(json_df['hours'])

In [131]:
test = json_df['attributes']

In [141]:
test.apply(lambda x: x is None or type(x) is dict).all()

True

In [128]:
test.apply(json.dumps)

0         "{\"BusinessAcceptsCreditCards\": \"True\", \"...
1         "{\"GoodForKids\": \"True\", \"ByAppointmentOn...
2                                                    "null"
3         "{\"BusinessAcceptsCreditCards\": \"True\", \"...
4         "{\"BusinessAcceptsCreditCards\": \"True\", \"...
                                ...                        
209388    "{\"Ambience\": \"{'romantic': False, 'intimat...
209389    "{\"RestaurantsPriceRange2\": \"1\", \"Busines...
209390    "{\"BusinessAcceptsCreditCards\": \"True\", \"...
209391                                               "null"
209392                                               "null"
Name: attributes, Length: 209393, dtype: object

In [106]:
json_df['attributes'].iloc[0]

'"{\\"BusinessAcceptsCreditCards\\": \\"True\\", \\"BikeParking\\": \\"True\\", \\"GoodForKids\\": \\"False\\", \\"BusinessParking\\": \\"{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}\\", \\"ByAppointmentOnly\\": \\"False\\", \\"RestaurantsPriceRange2\\": \\"3\\"}"'

In [83]:
str(json_df.to_records(index=False)[0]).replace("\'","\"")

'("f9NumwFMBDn751xgFiRbNA", "The Range At Lake Norman", "10913 Bailey Rd", "Cornelius", "NC", "28031", 35.4627242, -80.8526119, 3.5, 36, 1, {"BusinessAcceptsCreditCards": "True", "BikeParking": "True", "GoodForKids": "False", "BusinessParking": "{"garage": False, "street": False, "validated": False, "lot": True, "valet": False}", "ByAppointmentOnly": "False", "RestaurantsPriceRange2": "3"}, "Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping", {"Monday": "10:0-18:0", "Tuesday": "11:0-20:0", "Wednesday": "10:0-18:0", "Thursday": "11:0-20:0", "Friday": "11:0-20:0", "Saturday": "11:0-20:0", "Sunday": "13:0-18:0"})'

In [99]:
test = json_df['attributes'].iloc[0:2]

In [100]:
for x in test.iteritems():
    print(x)

(0, {'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'GoodForKids': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '3'})
(1, {'GoodForKids': 'True', 'ByAppointmentOnly': 'True'})


In [50]:
print(json_data[0])
print("\n\n")
print(json_list[0])

{'business_id': 'f9NumwFMBDn751xgFiRbNA', 'name': 'The Range At Lake Norman', 'address': '10913 Bailey Rd', 'city': 'Cornelius', 'state': 'NC', 'postal_code': '28031', 'latitude': 35.4627242, 'longitude': -80.8526119, 'stars': 3.5, 'review_count': 36, 'is_open': 1, 'attributes': {'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'GoodForKids': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '3'}, 'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping', 'hours': {'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0', 'Wednesday': '10:0-18:0', 'Thursday': '11:0-20:0', 'Friday': '11:0-20:0', 'Saturday': '11:0-20:0', 'Sunday': '13:0-18:0'}}



{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5

In [61]:
json.dumps(json_data[0])

'{"business_id": "f9NumwFMBDn751xgFiRbNA", "name": "The Range At Lake Norman", "address": "10913 Bailey Rd", "city": "Cornelius", "state": "NC", "postal_code": "28031", "latitude": 35.4627242, "longitude": -80.8526119, "stars": 3.5, "review_count": 36, "is_open": 1, "attributes": {"BusinessAcceptsCreditCards": "True", "BikeParking": "True", "GoodForKids": "False", "BusinessParking": "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", "ByAppointmentOnly": "False", "RestaurantsPriceRange2": "3"}, "categories": "Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping", "hours": {"Monday": "10:0-18:0", "Tuesday": "11:0-20:0", "Wednesday": "10:0-18:0", "Thursday": "11:0-20:0", "Friday": "11:0-20:0", "Saturday": "11:0-20:0", "Sunday": "13:0-18:0"}}'

In [5]:
#upload json to postgresSQL table
json_df.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None


In [21]:
#2 nested json objects : attributes and hours
#df_flattened_data = json_normalize(json_data,sep="_")
json_flattened_data = nested_to_record(json_data,sep="_",) #dont need to convert to df as destination is SQL database

In [23]:
#for flattening in future, may want to expand to convert nested json string to json
##see attributes_Business
json_flattened_data[0] 

{'business_id': 'f9NumwFMBDn751xgFiRbNA',
 'name': 'The Range At Lake Norman',
 'address': '10913 Bailey Rd',
 'city': 'Cornelius',
 'state': 'NC',
 'postal_code': '28031',
 'latitude': 35.4627242,
 'longitude': -80.8526119,
 'stars': 3.5,
 'review_count': 36,
 'is_open': 1,
 'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
 'attributes_BusinessAcceptsCreditCards': 'True',
 'attributes_BikeParking': 'True',
 'attributes_GoodForKids': 'False',
 'attributes_BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
 'attributes_ByAppointmentOnly': 'False',
 'attributes_RestaurantsPriceRange2': '3',
 'hours_Monday': '10:0-18:0',
 'hours_Tuesday': '11:0-20:0',
 'hours_Wednesday': '10:0-18:0',
 'hours_Thursday': '11:0-20:0',
 'hours_Friday': '11:0-20:0',
 'hours_Saturday': '11:0-20:0',
 'hours_Sunday': '13:0-18:0'}

### Storing JSON in PostgreSQL
- upon further research, it looks like we can store JSON directly in PostgreSQL without the traditional field format
    - [Storing JSON in PostgreSQL: A must-know feature] (https://www.blendo.co/blog/storing-json-in-postgresql/)
    - [Replacing EAV with JSONB in PostgreSQL*](https://coussej.github.io/2016/01/14/Replacing-EAV-with-JSONB-in-PostgreSQL/)

    *EAV = Entity,Attribute,Value (ie three tables connected with joins to get fields for the entity)


- find connection information 

>michaelkranz$ `psql`

>michaelkranz=# `CREATE DATABASE restaurants`

>michaelkranz=# `\c restaurants`

>restaurants=# `\conninfo`

> You are now connected to database "restaurants" as user "michaelkranz".
restaurants=# \conninfo
You are connected to database "restaurants" as user "michaelkranz" via socket in "/tmp" at port "5432".

- [CHAR and VARCHAR : no performance differences and character limits used to check and will return error if longer](https://www.postgresqltutorial.com/postgresql-char-varchar-text/)

- https://www.postgresql.org/docs/12/datatype-json.html

In [5]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL 
import psycopg2 #postgressql driver

In [6]:
postgres_db_params = {'drivername': 'postgres',
                'database':'restaurants',
               'username': 'michaelkranz',
               'password': 'helloworld',
               'host': 'localhost',
               'port': 5432}

postgres_db_url = URL(**postgres_db_params)

In [7]:
engine = create_engine(postgres_db_url)

In [38]:
# store first level as a column but hours and attributes as jsonb format
business_info_create_table_str = '''
CREATE TABLE business_info (
    business_id CHAR(22) PRIMARY KEY 
    ,address VARCHAR
    ,city VARCHAR
    ,state VARCHAR
    ,postal_code VARCHAR  
    ,latitude FLOAT
    ,longitude FLOAT
    ,stars FLOAT(1)
    ,review_count INTEGER  
    ,is_open BOOLEAN
    ,attributes JSONB
    ,categories VARCHAR
    ,hours JSONB
);
'''

In [39]:
with engine.connect() as conn:
    conn.execute(business_info_create_table_str)

ProgrammingError: (psycopg2.errors.DuplicateTable) relation "business_info" already exists

[SQL: 
CREATE TABLE business_info (
    business_id CHAR(22) PRIMARY KEY 
    ,address VARCHAR
    ,city VARCHAR
    ,state VARCHAR
    ,postal_code VARCHAR  
    ,latitude FLOAT
    ,longitude FLOAT
    ,stars FLOAT(1)
    ,review_count INTEGER  
    ,is_open BOOLEAN
    ,attributes JSONB
    ,categories VARCHAR
    ,hours JSONB
);
]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [71]:
#get 
(pd.read_sql(con=conn,sql='''
SELECT table_name
    ,column_name
    ,data_type
FROM information_schema.columns
WHERE
    table_name LIKE 'b%'
''')
.sort_values('table_name',ascending=True)
)

,table_name,column_name,data_type
0,business_info,business_id,character
1,business_info,address,character varying
2,business_info,city,character varying
3,business_info,state,character varying
4,business_info,postal_code,character varying
5,business_info,latitude,double precision
6,business_info,longitude,double precision
7,business_info,stars,real
8,business_info,review_count,integer
9,business_info,is_open,boolean


In [86]:
json_df.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [88]:
hours = json_df['hours']

In [90]:
hours.describe()

KeyboardInterrupt: 

In [ ]:
hours.str.replace("\''")

In [84]:
json_str_list = [
    sub("\'","\"",sub(",\'{|}\',",",{|},",str(x)))
    for x in json_df.to_records(index=False).tolist()
    ]

In [85]:
json_str_list[0]

'("f9NumwFMBDn751xgFiRbNA", "The Range At Lake Norman", "10913 Bailey Rd", "Cornelius", "NC", "28031", 35.4627242, -80.8526119, 3.5, 36, 1, {"BusinessAcceptsCreditCards": "True", "BikeParking": "True", "GoodForKids": "False", "BusinessParking": "{"garage": False, "street": False, "validated": False, "lot": True, "valet": False}", "ByAppointmentOnly": "False", "RestaurantsPriceRange2": "3"}, "Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping", {"Monday": "10:0-18:0", "Tuesday": "11:0-20:0", "Wednesday": "10:0-18:0", "Thursday": "11:0-20:0", "Friday": "11:0-20:0", "Saturday": "11:0-20:0", "Sunday": "13:0-18:0"})'

In [1]:
conn = engine.connect()

NameError: name 'engine' is not defined

In [9]:
business_info_insert_str = '''
INSERT INTO business_info
VALUES {}
'''

In [12]:
json_str_list[0]

'(\'f9NumwFMBDn751xgFiRbNA\', \'The Range At Lake Norman\', \'10913 Bailey Rd\', \'Cornelius\', \'NC\', \'28031\', 35.4627242, -80.8526119, 3.5, 36, 1, {\'BusinessAcceptsCreditCards\': \'True\', \'BikeParking\': \'True\', \'GoodForKids\': \'False\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'RestaurantsPriceRange2\': \'3\'}, \'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping\', {\'Monday\': \'10:0-18:0\', \'Tuesday\': \'11:0-20:0\', \'Wednesday\': \'10:0-18:0\', \'Thursday\': \'11:0-20:0\', \'Friday\': \'11:0-20:0\', \'Saturday\': \'11:0-20:0\', \'Sunday\': \'13:0-18:0\'})'

In [30]:
conn = engine.connect()

In [33]:
json_list[0]

'{"business_id":"f9NumwFMBDn751xgFiRbNA","name":"The Range At Lake Norman","address":"10913 Bailey Rd","city":"Cornelius","state":"NC","postal_code":"28031","latitude":35.4627242,"longitude":-80.8526119,"stars":3.5,"review_count":36,"is_open":1,"attributes":{"BusinessAcceptsCreditCards":"True","BikeParking":"True","GoodForKids":"False","BusinessParking":"{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}","ByAppointmentOnly":"False","RestaurantsPriceRange2":"3"},"categories":"Active Life, Gun\\/Rifle Ranges, Guns & Ammo, Shopping","hours":{"Monday":"10:0-18:0","Tuesday":"11:0-20:0","Wednesday":"10:0-18:0","Thursday":"11:0-20:0","Friday":"11:0-20:0","Saturday":"11:0-20:0","Sunday":"13:0-18:0"}}\n'

In [36]:
print('''
INSERT INTO business_info(hours)
VALUES (
'{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0', 'Wednesday': '10:0-18:0', 'Thursday': '11:0-20:0', 'Friday': '11:0-20:0', 'Saturday': '11:0-20:0', 'Sunday': '13:0-18:0'}');
''')


INSERT INTO business_info(hours)
VALUES (
'{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0', 'Wednesday': '10:0-18:0', 'Thursday': '11:0-20:0', 'Friday': '11:0-20:0', 'Saturday': '11:0-20:0', 'Sunday': '13:0-18:0'}');



In [41]:
conn = engine.connect()

In [45]:
conn.execute('''
INSERT INTO business_info(hours)
VALUES (
\'{'test':1}\'
);
'''
)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "test"
LINE 4: '{'test':1}'
           ^

[SQL: 
INSERT INTO business_info(hours)
VALUES (
'{'test':1}'
);
]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [15]:
test_insert_str = business_info_insert_str.format(json_str_list[0])
print(test_insert_str)


INSERT INTO business_info
VALUES ('f9NumwFMBDn751xgFiRbNA', 'The Range At Lake Norman', '10913 Bailey Rd', 'Cornelius', 'NC', '28031', 35.4627242, -80.8526119, 3.5, 36, 1, {'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'GoodForKids': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '3'}, 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping', {'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0', 'Wednesday': '10:0-18:0', 'Thursday': '11:0-20:0', 'Friday': '11:0-20:0', 'Saturday': '11:0-20:0', 'Sunday': '13:0-18:0'})



In [17]:
with engine.connect() as conn:
    conn.execute(test_insert_str)

ProgrammingError: (psycopg2.errors.SyntaxError) syntax error at or near "{"
LINE 3: ...C', '28031', 35.4627242, -80.8526119, 3.5, 36, 1, {'Business...
                                                             ^

[SQL: 
INSERT INTO business_info
VALUES ('f9NumwFMBDn751xgFiRbNA', 'The Range At Lake Norman', '10913 Bailey Rd', 'Cornelius', 'NC', '28031', 35.4627242, -80.8526119, 3.5, 36, 1, {'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'GoodForKids': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '3'}, 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping', {'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0', 'Wednesday': '10:0-18:0', 'Thursday': '11:0-20:0', 'Friday': '11:0-20:0', 'Saturday': '11:0-20:0', 'Sunday': '13:0-18:0'})
]
(Background on this error at: http://sqlalche.me/e/13/f405)

In [ ]:
with engine.connect() as conn:
    for json_str in json_str_list:
        conn.execute(
            business_info_insert_str(json_str)
        )

In [122]:
INSERT INTO business_info
VALUES ({})

[{'business_id': 'f9NumwFMBDn751xgFiRbNA',
  'name': 'The Range At Lake Norman',
  'address': '10913 Bailey Rd',
  'city': 'Cornelius',
  'state': 'NC',
  'postal_code': '28031',
  'latitude': 35.4627242,
  'longitude': -80.8526119,
  'stars': 3.5,
  'review_count': 36,
  'is_open': 1,
  'attributes': {'BusinessAcceptsCreditCards': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'False',
   'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
   'ByAppointmentOnly': 'False',
   'RestaurantsPriceRange2': '3'},
  'categories': 'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping',
  'hours': {'Monday': '10:0-18:0',
   'Tuesday': '11:0-20:0',
   'Wednesday': '10:0-18:0',
   'Thursday': '11:0-20:0',
   'Friday': '11:0-20:0',
   'Saturday': '11:0-20:0',
   'Sunday': '13:0-18:0'}}]

In [117]:
test.format(x for x in json_df.iloc[0:1].iteritems())

'\nINSERT INTO business_info \nVALUES (\n    <generator object <genexpr> at 0x19e36bd50>\n    )\n    '

In [104]:
json_df.to_sql("business_info",if_exists='replace',con=conn,index=False)

ProgrammingError: (psycopg2.ProgrammingError) can't adapt type 'dict'
[SQL: INSERT INTO business_info (business_id, name, address, city, state, postal_code, latitude, longitude, stars, review_count, is_open, attributes, categories, hours) VALUES (%(business_id)s, %(name)s, %(address)s, %(city)s, %(state)s, %(postal_code)s, %(latitude)s, %(longitude)s, %(stars)s, %(review_count)s, %(is_open)s, %(attributes)s, %(categories)s, %(hours)s)]
[parameters: ({'business_id': 'f9NumwFMBDn751xgFiRbNA', 'name': 'The Range At Lake Norman', 'address': '10913 Bailey Rd', 'city': 'Cornelius', 'state': 'NC', 'postal_code': '28031', 'latitude': 35.4627242, 'longitude': -80.8526119, 'stars': 3.5, 'review_count': 36, 'is_open': 1, 'attributes': {'BusinessAcceptsCreditCards': 'True', 'BikeParking': 'True', 'GoodForKids': 'False', 'BusinessParking': "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}", 'ByAppointmentOnly': 'False', 'RestaurantsPriceRange2': '3'}, 'categories'

In [105]:
pd.read_sql(con=conn,sql="SELECT * FROM business_info")

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours


In [93]:
test = '''
INSERT INTO business_info 
VALUES (
    {}
    )
    '''

In [95]:
test.format(json_data[0])

'\nINSERT INTO business_info \nVALUES (\n    {\'business_id\': \'f9NumwFMBDn751xgFiRbNA\', \'name\': \'The Range At Lake Norman\', \'address\': \'10913 Bailey Rd\', \'city\': \'Cornelius\', \'state\': \'NC\', \'postal_code\': \'28031\', \'latitude\': 35.4627242, \'longitude\': -80.8526119, \'stars\': 3.5, \'review_count\': 36, \'is_open\': 1, \'attributes\': {\'BusinessAcceptsCreditCards\': \'True\', \'BikeParking\': \'True\', \'GoodForKids\': \'False\', \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'ByAppointmentOnly\': \'False\', \'RestaurantsPriceRange2\': \'3\'}, \'categories\': \'Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping\', \'hours\': {\'Monday\': \'10:0-18:0\', \'Tuesday\': \'11:0-20:0\', \'Wednesday\': \'10:0-18:0\', \'Thursday\': \'11:0-20:0\', \'Friday\': \'11:0-20:0\', \'Saturday\': \'11:0-20:0\', \'Sunday\': \'13:0-18:0\'}}\n    )\n    '

In [126]:
json_df.iloc[0]

business_id                                f9NumwFMBDn751xgFiRbNA
name                                     The Range At Lake Norman
address                                           10913 Bailey Rd
city                                                    Cornelius
state                                                          NC
postal_code                                                 28031
latitude                                                  35.4627
longitude                                                -80.8526
stars                                                         3.5
review_count                                                   36
is_open                                                         1
attributes      {'BusinessAcceptsCreditCards': 'True', 'BikePa...
categories      Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...
hours           {'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'...
Name: 0, dtype: object